In [1]:
import os
import shutil

# 정리할 폴더 경로
folder_paths = [
    'C:\\Users\\ohbok\\Taipy\\sparc-malfunction-classification-develop\\src\\.taipy',
    'C:\\Users\\ohbok\\Taipy\\sparc-malfunction-classification-develop\\src\\user_data'
]

for path in folder_paths:
    # 폴더가 존재하는지 확인
    if os.path.exists(path) and os.path.isdir(path):
        try:
            # 폴더 하위의 모든 항목을 순회하며 삭제
            for item in os.listdir(path):
                item_path = os.path.join(path, item)
                if os.path.isdir(item_path):
                    # 하위 폴더인 경우 shutil.rmtree로 삭제
                    shutil.rmtree(item_path)
                else:
                    # 파일인 경우 os.remove로 삭제
                    os.remove(item_path)
            print(f"폴더 '{path}'의 하위 내용이 성공적으로 삭제되었습니다.")
        except OSError as e:
            print(f"폴더 '{path}'의 하위 내용 삭제 중 오류가 발생했습니다: {e}")
    else:
        print(f"폴더 '{path}'가 존재하지 않거나 디렉토리가 아닙니다.")

폴더 'C:\Users\ohbok\Taipy\sparc-malfunction-classification-develop\src\.taipy'의 하위 내용이 성공적으로 삭제되었습니다.
폴더 'C:\Users\ohbok\Taipy\sparc-malfunction-classification-develop\src\user_data'의 하위 내용이 성공적으로 삭제되었습니다.


In [2]:
# 데이터 처리 및 기타 작업을 위한 라이브러리들을 불러옵니다.
import pandas as pd                                         # 데이터프레임 구조를 다루는 데 필수적인 라이브러리
import numpy as np                                          # 숫자 연산을 위한 라이브러리
import datetime as dt                                       # 날짜와 시간을 다루는 라이브러리
import json                                                 # JSON 형식의 데이터를 처리하는 라이브러리

import optuna
import pprint

import taipy as tp
from taipy import Config, Orchestrator, Gui

In [3]:
# # Ensure the orchestrator is stopped before modifying the configuration
tp.Orchestrator().stop()

[2025-08-25 14:54:52.186][Taipy][INFO] Unblocking configuration update...
[2025-08-25 14:54:52.186][Taipy][INFO] Stopping job dispatcher...
[2025-08-25 14:54:52.187][Taipy][INFO] Orchestrator service has been stopped.


In [4]:

from config.config import scenario_cfg

In [5]:
Config.configure_job_executions(mode="standalone", max_nb_of_workers=2)
tp.Orchestrator().run()


[2025-08-25 14:54:55.638][Taipy][INFO] Updating configuration with command-line arguments...
[2025-08-25 14:54:55.639][Taipy][INFO] Managing application's version...
[2025-08-25 14:54:55.648][Taipy][INFO] Development mode: Clean all entities of version 449c2e84-2416-44eb-9b2e-ec4783a66dc4
[2025-08-25 14:54:55.666][Taipy][INFO] Checking application's version...
[2025-08-25 14:54:55.669][Taipy][INFO] Blocking configuration update...
[2025-08-25 14:54:55.670][Taipy][INFO] Starting job dispatcher...
[2025-08-25 14:54:55.673][Taipy][INFO] Orchestrator service has been started.


In [6]:
from algos.algos import *

In [7]:
# Config.load(r"/usr01/automl/sparc-malfunction-classification-develop/src/config/config.toml")
# scenario_cfg = Config.scenarios["churn_classification"]

scenario_1 = tp.create_scenario(scenario_cfg)

SCENARIO

In [8]:
# # scenario_1.submit(wait=True, timeout=300)
# scenario_1.submit(wait=True)

In [9]:
# train_dataset = scenario_1.train_dataset.read()
# feature_selection_info = scenario_1.feature_selection_info.read()
# train_parameters = scenario_1.train_parameters_random_forest_optuna.read()

SEQUENCE

TASK

In [10]:
# train_parameters_default = scenario_1.train_parameters_list_default.read()
# pprint.pprint(train_parameters_default)

In [11]:
initial_dataset = scenario_1.initial_dataset.read()

In [12]:
print(initial_dataset.shape)
# initial_dataset

(4536, 1625)


In [13]:
task = scenario_1.preprocess_dataset
# task.submit(wait=True, timeout=50)
task.submit(wait=True)

[2025-08-25 14:55:08.135][Taipy][INFO] job JOB_preprocess_dataset_0d6b5232-9a37-4774-a62c-0fa379a159fd is completed.


In [14]:
preprocessed_dataset = scenario_1.preprocessed_dataset.read()
print(preprocessed_dataset.shape)

(4536, 1400)


In [15]:
task = scenario_1.create_train_and_test_data
# task.submit(wait=True, timeout=300)
task.submit(wait=True)

[2025-08-25 14:55:13.648][Taipy][INFO] job JOB_create_train_and_test_data_cd6d5d11-33cd-4eaf-b5ff-736b3abb38ce is completed.


In [16]:
split_parameter_info = scenario_1.split_parameter_info.read()
# pprint.pprint(split_parameter_info)

In [17]:
train_dataset = scenario_1.train_dataset.read()
print(train_dataset.shape)
test_dataset = scenario_1.test_dataset.read()
print(test_dataset.shape)

(3617, 1400)
(905, 1400)


In [18]:
# train_dataset

In [19]:
# test_dataset

variance_filter
start

In [20]:
# variance_filter = \
# {
#     "feature_selector_name": "FeatureFilter",
#     "filter_methods": {
#         "apply_variance_filter": True,
#         "var_threshold": 0.00,
#         "apply_target_linear_corr_filter": False,
#         "target_linear_corr_threshold": 0.5,
#         "apply_target_xicor_filter": False,
#         "target_xicor_threshold": 0.5,
#         "apply_feature_linear_corr_filter": False,
#         "feature_linear_corr_threshold": 0.95,
#         "apply_feature_xicor_filter": False,
#         "feature_xicor_threshold": 0.9
#     }
# }

In [21]:
feature_selector = scenario_1.feature_selector_params_FeatureFilter_default.read()
# feature_selector = scenario_1.feature_selector_params_SelectFromModel_default.read()

feature_selector["filter_methods"]["apply_variance_filter"] = True
feature_selector["filter_methods"]["var_threshold"] = 0.00
# pprint.pprint(feature_selector)

In [22]:
scenario_1.feature_selector.write(feature_selector)
pprint.pprint(scenario_1.feature_selector.read())

task = scenario_1.select_feature
task.submit(wait=True)

{'feature_selector_name': 'FeatureFilter',
 'filter_methods': {'apply_feature_linear_corr_filter': False,
                    'apply_feature_xicor_filter': False,
                    'apply_target_linear_corr_filter': False,
                    'apply_target_xicor_filter': False,
                    'apply_variance_filter': True,
                    'feature_linear_corr_threshold': 0.95,
                    'feature_xicor_threshold': 0.9,
                    'target_linear_corr_threshold': 0.05,
                    'target_xicor_threshold': 0.1,
                    'var_threshold': 0.0}}
[2025-08-25 14:55:17.395][Taipy][INFO] job JOB_select_feature_aa175904-6b55-46de-89db-7193e025b876 is completed.


In [23]:
feature_selection_info = scenario_1.feature_selection_info.read()
feature_selection_info["final_feature_count"]
# pprint.pprint(feature_selection_info)
feature_selection_variance = feature_selection_info

In [24]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]
# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
train_dataset_filtered_variance = train_dataset[columns_to_keep]

print(train_dataset_filtered_variance.shape)

(3617, 1398)


variance_filter
end

target_linear_corr_filter
start

In [25]:
# target_linear_corr_filter = \
# {
#     "feature_selector_name": "FeatureFilter",
#     "filter_methods": {
#         "apply_variance_filter": False,
#         "var_threshold": 0.01,
#         "apply_target_linear_corr_filter": True,
#         "target_linear_corr_threshold": 0.00,
#         "apply_target_xicor_filter": False,
#         "target_xicor_threshold": 0.5,
#         "apply_feature_linear_corr_filter": False,
#         "feature_linear_corr_threshold": 0.95,
#         "apply_feature_xicor_filter": False,
#         "feature_xicor_threshold": 0.9
#     }
# }

In [26]:
feature_selector = scenario_1.feature_selector_params_FeatureFilter_default.read()
# feature_selector = scenario_1.feature_selector_params_SelectFromModel_default.read()

feature_selector["filter_methods"]["apply_target_linear_corr_filter"] = True
feature_selector["filter_methods"]["target_linear_corr_threshold"] = 0.00
# pprint.pprint(feature_selector)

In [27]:
scenario_1.feature_selector.write(feature_selector)
pprint.pprint(scenario_1.feature_selector.read())

task = scenario_1.select_feature
task.submit(wait=True)

{'feature_selector_name': 'FeatureFilter',
 'filter_methods': {'apply_feature_linear_corr_filter': False,
                    'apply_feature_xicor_filter': False,
                    'apply_target_linear_corr_filter': True,
                    'apply_target_xicor_filter': False,
                    'apply_variance_filter': False,
                    'feature_linear_corr_threshold': 0.95,
                    'feature_xicor_threshold': 0.9,
                    'target_linear_corr_threshold': 0.0,
                    'target_xicor_threshold': 0.1,
                    'var_threshold': 0.01}}
[2025-08-25 14:55:20.877][Taipy][INFO] job JOB_select_feature_1d76005e-cfd2-4e12-bd93-8644420861ed is completed.


In [28]:
feature_selection_info = scenario_1.feature_selection_info.read()
feature_selection_info["final_feature_count"]
# pprint.pprint(feature_selection_info)
feature_selection_target_linear_corr = feature_selection_info

In [29]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]
# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
train_dataset_target_linear_corr_filter = train_dataset[columns_to_keep]

print(train_dataset_target_linear_corr_filter.shape)

(3617, 1400)


target_linear_corr_filter
end

target_xicor_filter
start

In [30]:
# apply_target_xicor_filter = \
# {
#     "feature_selector_name": "FeatureFilter",
#     "filter_methods": {
#         "apply_variance_filter": False,
#         "var_threshold": 0.01,
#         "apply_target_linear_corr_filter": False,
#         "target_linear_corr_threshold": 0.01,
#         "apply_target_xicor_filter": True,
#         "target_xicor_threshold": 0.0,
#         "apply_feature_linear_corr_filter": False,
#         "feature_linear_corr_threshold": 0.95,
#         "apply_feature_xicor_filter": False,
#         "feature_xicor_threshold": 0.9
#     }
# }

In [31]:
train_dataset_part = train_dataset.iloc[:, -5:]
scenario_1.train_dataset.write(train_dataset_part)

In [32]:
feature_selector = scenario_1.feature_selector_params_FeatureFilter_default.read()
# feature_selector = scenario_1.feature_selector_params_SelectFromModel_default.read()

feature_selector["filter_methods"]["apply_target_xicor_filter"] = True
feature_selector["filter_methods"]["target_xicor_threshold"] = 0.00
# pprint.pprint(feature_selector)

In [33]:
scenario_1.feature_selector.write(feature_selector)
pprint.pprint(scenario_1.feature_selector.read())

task = scenario_1.select_feature
task.submit(wait=True)

{'feature_selector_name': 'FeatureFilter',
 'filter_methods': {'apply_feature_linear_corr_filter': False,
                    'apply_feature_xicor_filter': False,
                    'apply_target_linear_corr_filter': False,
                    'apply_target_xicor_filter': True,
                    'apply_variance_filter': False,
                    'feature_linear_corr_threshold': 0.95,
                    'feature_xicor_threshold': 0.9,
                    'target_linear_corr_threshold': 0.05,
                    'target_xicor_threshold': 0.0,
                    'var_threshold': 0.01}}
[2025-08-25 14:55:36.552][Taipy][INFO] job JOB_select_feature_c6fbe5d6-537b-40a0-aeb6-921c474c311e is completed.


In [34]:
feature_selection_info = scenario_1.feature_selection_info.read()
feature_selection_info["final_feature_count"]
# pprint.pprint(feature_selection_info)
feature_selection_target_xicor = feature_selection_info

In [35]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]
# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
train_dataset_target_xicor_filter = train_dataset[columns_to_keep]

print(train_dataset_target_xicor_filter.shape)

(3617, 5)


target_xicor_filter
end

feature_linear_corr_filter
start

In [36]:
# feature_linear_corr_filter = \
# {
#     "feature_selector_name": "FeatureFilter",
#     "filter_methods": {
#         "apply_variance_filter": False,
#         "var_threshold": 0.01,
#         "apply_target_linear_corr_filter": False,
#         "target_linear_corr_threshold": 0.00,
#         "apply_target_xicor_filter": False,
#         "target_xicor_threshold": 0.5,
#         "apply_feature_linear_corr_filter": True,
#         "feature_linear_corr_threshold": 0.95,
#         "apply_feature_xicor_filter": False,
#         "feature_xicor_threshold": 0.9
#     }
# }

In [37]:
scenario_1.train_dataset.write(train_dataset)

In [38]:
feature_selector = scenario_1.feature_selector_params_FeatureFilter_default.read()
# feature_selector = scenario_1.feature_selector_params_SelectFromModel_default.read()

feature_selector["filter_methods"]["apply_feature_linear_corr_filter"] = True
feature_selector["filter_methods"]["feature_linear_corr_threshold"] = 0.95
# pprint.pprint(feature_selector)

In [ ]:
scenario_1.feature_selector.write(feature_selector)
pprint.pprint(scenario_1.feature_selector.read())

task = scenario_1.select_feature
task.submit(wait=True)

{'feature_selector_name': 'FeatureFilter',
 'filter_methods': {'apply_feature_linear_corr_filter': True,
                    'apply_feature_xicor_filter': False,
                    'apply_target_linear_corr_filter': False,
                    'apply_target_xicor_filter': False,
                    'apply_variance_filter': False,
                    'feature_linear_corr_threshold': 0.95,
                    'feature_xicor_threshold': 0.9,
                    'target_linear_corr_threshold': 0.05,
                    'target_xicor_threshold': 0.1,
                    'var_threshold': 0.01}}


In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()
feature_selection_info["final_feature_count"]
# pprint.pprint(feature_selection_info)
feature_selection_feature_linear_corr = feature_selection_info

In [ ]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]
# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
feature_linear_corr_filter = train_dataset[columns_to_keep]

print(feature_linear_corr_filter.shape)
# feature_linear_corr_filter

(3617, 607)


feature_linear_corr_filter
end

feature_xicor_filter
start

In [ ]:
# feature_xicor_filter = \
# {
#     "feature_selector_name": "FeatureFilter",
#     "filter_methods": {
#         "apply_variance_filter": False,
#         "var_threshold": 0.01,
#         "apply_target_linear_corr_filter": False,
#         "target_linear_corr_threshold": 0.00,
#         "apply_target_xicor_filter": False,
#         "target_xicor_threshold": 0.5,
#         "apply_feature_linear_corr_filter": False,
#         "feature_linear_corr_threshold": 0.95,
#         "apply_feature_xicor_filter": True,
#         "feature_xicor_threshold": 0.9
#     }
# }

In [ ]:
train_dataset_part = train_dataset.iloc[:, -7:]
scenario_1.train_dataset.write(train_dataset_part)

In [ ]:
feature_selector = scenario_1.feature_selector_params_FeatureFilter_default.read()
# feature_selector = scenario_1.feature_selector_params_SelectFromModel_default.read()

feature_selector["filter_methods"]["apply_feature_xicor_filter"] = True
feature_selector["filter_methods"]["feature_xicor_threshold"] = 0.9
# pprint.pprint(feature_selector)

In [ ]:
scenario_1.feature_selector.write(feature_selector)
pprint.pprint(scenario_1.feature_selector.read())

task = scenario_1.select_feature
task.submit(wait=True)

{'feature_selector_name': 'FeatureFilter',
 'filter_methods': {'apply_feature_linear_corr_filter': False,
                    'apply_feature_xicor_filter': True,
                    'apply_target_linear_corr_filter': False,
                    'apply_target_xicor_filter': False,
                    'apply_variance_filter': False,
                    'feature_linear_corr_threshold': 0.95,
                    'feature_xicor_threshold': 0.9,
                    'target_linear_corr_threshold': 0.05,
                    'target_xicor_threshold': 0.1,
                    'var_threshold': 0.01}}
[2025-08-25 14:03:43.576][Taipy][INFO] job JOB_select_feature_d0bb6ef6-9d8c-4c74-a56d-b2ba9c8ac59a is completed.


In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()
feature_selection_info["final_feature_count"]
# pprint.pprint(feature_selection_info)
feature_selection_feature_xicor = feature_selection_info

In [ ]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]
# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
train_dataset_feature_xicor_filter = train_dataset[columns_to_keep]

print(train_dataset_feature_xicor_filter.shape)

(3617, 7)


feature_xicor_filter
end

feature_SelectFromModel
start

In [ ]:
scenario_1.train_dataset.write(train_dataset)

In [ ]:
# # train_dataset_filtered_variance df의 앞 10개와 맨 뒤 1개 컬럼을 선택
# cols_to_keep = train_dataset_filtered_variance.columns[:100].tolist()
# cols_to_keep.append(train_dataset_filtered_variance.columns[-1])

# # 선택된 컬럼들로 새로운 데이터프레임 생성
# train_dataset_filtered_variance_part = train_dataset_filtered_variance[cols_to_keep]

# # 결과를 확인하기 위해 새로운 데이터프레임의 컬럼 출력
# print("선택된 컬럼:")
# print(train_dataset_filtered_variance_part.columns)

# scenario_1.train_dataset.write(train_dataset_filtered_variance_part)

In [ ]:
# Embedded Method (SelectFromModel)를 적용하는 경우
feature_selector_params_SelectFromModel = \
{
    "feature_selector_name": "EmbeddedMethod",
    "model_based": {
        "threshold": "median",
        # "threshold": "1.25*median",
        # "threshold": "mean",
        # "threshold": "1.25*mean",
        "random_forest": {
            "n_estimators": 250,
            "max_depth": 12
        }
    }
}

In [ ]:
scenario_1.feature_selector.write(feature_selector_params_SelectFromModel)
feature_selector = scenario_1.feature_selector.read()
pprint.pprint(feature_selector)

{'feature_selector_name': 'EmbeddedMethod',
 'model_based': {'random_forest': {'max_depth': 12, 'n_estimators': 250},
                 'threshold': 'median'}}


In [ ]:
task = scenario_1.select_feature
task.submit(wait=True)

[2025-08-25 14:05:06.228][Taipy][INFO] job JOB_select_feature_96f53aea-582c-4422-bfc2-85c0efe1141b is completed.


In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()
# pprint.pprint(feature_selection_info)
# feature_selection_info
feature_selection_FromModel = feature_selection_info

In [ ]:
feature_selection_info["final_feature_count"]

700

In [ ]:
len(feature_selection_info["filter_details"]["feature_importances"])

1399

In [ ]:
# 'final_features' 리스트와 마지막 컬럼 이름을 포함하는 리스트 생성
columns_to_keep = feature_selection_info["final_features"] + [train_dataset.columns[-1]]
# 새로운 리스트에 있는 컬럼들만 남겨 DataFrame 업데이트
train_dataset_feature_SelectFromModel = train_dataset[columns_to_keep]

print(train_dataset_feature_SelectFromModel.shape)

(3617, 701)


feature_SelectFromModel
end

summary feature info
start

In [ ]:
# feature_selection_variance["filter_details"]["variance"]["features_values_checked"]

In [ ]:
value_type = "variance"

features_values_checked = feature_selection_variance["filter_details"][value_type]["features_values_checked"]
feature_selection_variance_df = pd.DataFrame(
    list(features_values_checked.items()), 
    columns=['feature_name', value_type]
)
feature_selection_variance_df.set_index('feature_name', inplace=True)
feature_selection_variance_df

In [ ]:
# feature_selection_target_linear_corr["filter_details"]

In [ ]:
value_type = "target_linear_correlation"

features_values_checked = feature_selection_target_linear_corr["filter_details"][value_type]["features_values_checked"]
feature_selection_target_linear_corr_df = pd.DataFrame(
    list(features_values_checked.items()), 
    columns=['feature_name', value_type]
)
feature_selection_target_linear_corr_df.set_index('feature_name', inplace=True)
feature_selection_target_linear_corr_df

In [ ]:
# feature_selection_target_xicor["filter_details"]

{'target_xicor_correlation': {'start_time': '2025-08-25 14:01:01',
  'end_time': '2025-08-25 14:01:13',
  'duration': '0:00:11',
  'threshold_value': 0.0,
  'original_count': 4,
  'remaining_count': 4,
  'features_dropped_yn': {'FAILING_BINOUTS(1) of 1103959_69_JPP_GbPoPo': 'False',
   'FAILING_BINOUTS(1) of 1103959_69_JPP_PoPo': 'False',
   'FAILING_BINOUTS(1) of 1103959_69_JPP_SbPoPo': 'False',
   'band gap dpat_ok for band gap': 'False'},
  'features_values_checked': {'FAILING_BINOUTS(1) of 1103959_69_JPP_GbPoPo': -7.717543328636047,
   'FAILING_BINOUTS(1) of 1103959_69_JPP_PoPo': -7.749440557093635,
   'FAILING_BINOUTS(1) of 1103959_69_JPP_SbPoPo': -7.7435821305282,
   'band gap dpat_ok for band gap': -7.7218459904704115}}}

In [ ]:
value_type = "target_xicor_correlation"

features_values_checked = feature_selection_target_xicor["filter_details"][value_type]["features_values_checked"]
feature_selection_target_xicor_df = pd.DataFrame(
    list(features_values_checked.items()), 
    columns=['feature_name', value_type]
)
feature_selection_target_xicor_df.set_index('feature_name', inplace=True)
feature_selection_target_xicor_df

In [ ]:
# feature_selection_FromModel["filter_details"]

# value_type = "feature_importances"
# feature_selection_FromModel["filter_details"][value_type]

In [ ]:
value_type = "feature_importances"

features_values_checked = feature_selection_FromModel["filter_details"][value_type]
feature_selection_FromModel_df = pd.DataFrame(
    list(features_values_checked.items()), 
    columns=['feature_name', value_type]
)
feature_selection_FromModel_df.set_index('feature_name', inplace=True)
# feature_selection_FromModel_df

In [ ]:

# 첫 번째와 두 번째 데이터프레임을 병합
feature_selection_XY_sum_df = pd.merge(
    feature_selection_variance_df,
    feature_selection_target_linear_corr_df,
    how='outer',
    left_index=True,
    right_index=True
)

# 세 번째 데이터프레임을 병합된 결과에 추가 병합
feature_selection_XY_sum_df = pd.merge(
    feature_selection_XY_sum_df,
    feature_selection_FromModel_df,
    how='outer',
    left_index=True,
    right_index=True
)


# 결과 출력
# print(feature_selection_XY_sum_df)

# feature_selection_XY_sum_df

,variance,target_linear_correlation,feature_importances
feature_name,,,
AC_GAIN_32 of 1103959_69_1133529_cp1_cp1p5_YPP,8.620564,0.114818,0.000000
AC_GAIN_32 of 1103959_69_1133529_cp1_cp1p5_YPP_QPP_RPP,8.620564,0.114818,0.000000
AC_GAIN_32 of 1103959_69_1133529_cp1p5,8.620564,0.114818,0.000000
AC_GAIN_32 of 1103959_69_1133592_QPP,8.620564,0.114818,0.000000
AC_OFFCALRES of 1103959_69_1133529_cp1,0.367364,0.001054,0.001189
...,...,...,...
Zb_V_OBVOL_VSS_L of 1103959_69_1133529_cp1_cp1p5_YPP_QPP_RPP,0.000030,0.017764,0.000161
Zb_V_OBVOL_VSS_L of 1103959_69_1133529_cp1p5,0.000047,0.026030,0.000903
Zb_V_OBVOL_VSS_L of 1103959_69_1133592_QPP,0.000037,0.001384,0.000263


In [ ]:
#@@@@

In [ ]:
task = scenario_1.select_feature
task.submit(wait=True)

In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()

In [ ]:
feature_selection_info

In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()
# pprint.pprint(feature_selection_info)
pprint.pprint(feature_selection_info.get('final_feature_count'))

### TASKS train_model

In [ ]:
# Iterate through all scenarios and models
for scenario in tp.get_scenarios():
    for task_name in scenario.tasks:
        if "train_model" in task_name:
            model = task_name.split("train_model_")[1]
            print(f"Model: {model}")
            task = eval("scenario." + task_name)
            task.submit(wait=True)

In [ ]:
task = scenario_1.train_model_baseline
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
trained_model_baseline = scenario_1.trained_model_baseline.read()
print(trained_model_baseline)
feature_importance_baseline = scenario_1.feature_importance_baseline.read()
print(feature_importance_baseline.shape)

In [ ]:
# train_parameters_logistic_regression = scenario_1.train_parameters_logistic_regression.read()
# train_parameters_logistic_regression

In [ ]:
task = scenario_1.train_model_logistic_regression
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
trained_model_logistic_regression = scenario_1.trained_model_logistic_regression.read()
print(trained_model_logistic_regression)
feature_importance_logistic_regression = scenario_1.feature_importance_logistic_regression.read()
print(feature_importance_logistic_regression.shape)

In [ ]:
# train_parameters_info_logistic_regression = scenario_1.train_parameters_info_logistic_regression.read()
# pprint.pprint(train_parameters_info_logistic_regression)

In [ ]:
# trained_model_logistic_regression
# feature_importance_logistic_regression
# train_parameters_info_logistic_regression

테스트 

In [ ]:
train_dataset = scenario_1.train_dataset.read()
# train_dataset

In [ ]:
feature_selection_info = scenario_1.feature_selection_info.read()


In [ ]:
# train_parameters_logistic_regression = scenario_1.train_parameters_logistic_regression.read()
# train_parameters = train_parameters_logistic_regression
# train_parameters

In [ ]:
# train_parameters_random_forest = scenario_1.train_parameters_random_forest.read()
# train_parameters = train_parameters_random_forest
# train_parameters

In [ ]:
task = scenario_1.train_model_random_forest
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
trained_model_random_forest = scenario_1.trained_model_random_forest.read()
print(trained_model_random_forest)
feature_importance_random_forest = scenario_1.feature_importance_random_forest.read()
print(feature_importance_random_forest.shape)

In [ ]:
task = scenario_1.train_model_xgboost
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
trained_model_xgboost = scenario_1.trained_model_xgboost.read()
print(trained_model_xgboost)
feature_importance_xgboost = scenario_1.feature_importance_xgboost.read()
print(feature_importance_xgboost.shape)

### TASKS predict_the_test_data

In [ ]:
task = scenario_1.predict_the_test_data_baseline
# task.submit(wait=True, timeout=50)
task.submit(wait=True, timeout=1)
forecast_dataset_baseline = scenario_1.forecast_dataset_baseline.read()
print(forecast_dataset_baseline)
shap_values_baseline = scenario_1.shap_values_baseline.read()


In [ ]:
# forecast_dataset_baseline

In [ ]:
# shap_values_baseline이 NoneType인 경우를 테스트하기 위한 예시
# 아래 코드를 주석 처리하고 실제 shap_values_baseline을 사용하세요.
# shap_values_baseline = (None, np.random.rand(100, 5))
# 조건문
if isinstance(shap_values_baseline[0], type(None)):
    print(type(shap_values_baseline[0]))
    print(shap_values_baseline[1].shape)
else:
    shap.summary_plot(shap_values_baseline[0], shap_values_baseline[1])

In [ ]:
task = scenario_1.predict_the_test_data_logistic_regression
# task.submit(wait=True, timeout=50)
task.submit(wait=True)
forecast_dataset_logistic_regression = scenario_1.forecast_dataset_logistic_regression.read()

In [ ]:
print(forecast_dataset_logistic_regression)
shap_values_logistic_regression = scenario_1.shap_values_logistic_regression.read()


In [ ]:

# shap_values_logistic_regression이 NoneType인 경우를 테스트하기 위한 예시
# 아래 코드를 주석 처리하고 실제 shap_values_logistic_regression을 사용하세요.
# shap_values_logistic_regression = (None, np.random.rand(100, 5))
# 조건문
if isinstance(shap_values_logistic_regression[0], type(None)):
    print(type(shap_values_logistic_regression[0]))
    print(shap_values_logistic_regression[1].shape)
else:
    shap.summary_plot(shap_values_logistic_regression[0], shap_values_logistic_regression[1])

In [ ]:
task = scenario_1.predict_the_test_data_random_forest
task.submit(wait=True, timeout=50)
forecast_dataset_random_forest = scenario_1.forecast_dataset_random_forest.read()
print(forecast_dataset_random_forest)
shap_values_random_forest = scenario_1.shap_values_random_forest.read()

# shap_values_random_forest이 NoneType인 경우를 테스트하기 위한 예시
# 아래 코드를 주석 처리하고 실제 shap_values_random_forest을 사용하세요.
# shap_values_random_forest = (None, np.random.rand(100, 5))
# 조건문
if isinstance(shap_values_random_forest[0], type(None)):
    print(type(shap_values_random_forest[0]))
    print(shap_values_random_forest[1].shape)
else:
    shap.summary_plot(shap_values_random_forest[0], shap_values_random_forest[1])

In [ ]:
task = scenario_1.predict_the_test_data_xgboost
task.submit(wait=True, timeout=50)
forecast_dataset_xgboost = scenario_1.forecast_dataset_xgboost.read()
print(forecast_dataset_xgboost)
shap_values_xgboost = scenario_1.shap_values_xgboost.read()

# shap_values_xgboost이 NoneType인 경우를 테스트하기 위한 예시
# 아래 코드를 주석 처리하고 실제 shap_values_xgboost을 사용하세요.
# shap_values_xgboost = (None, np.random.rand(100, 5))
# 조건문
if isinstance(shap_values_xgboost[0], type(None)):
    print(type(shap_values_xgboost[0]))
    print(shap_values_xgboost[1].shape)
else:
    shap.summary_plot(shap_values_xgboost[0], shap_values_xgboost[1])

### TASKS find_best_threshold

In [ ]:
task = scenario_1.find_best_threshold_baseline
task.submit(wait=True, timeout=50)
train_dataset_proba_baseline = scenario_1.train_dataset_proba_baseline.read()
print(train_dataset_proba_baseline.shape)
threshold_baseline = scenario_1.threshold_baseline.read()
print(threshold_baseline)

In [ ]:
task = scenario_1.find_best_threshold_logistic_regression
task.submit(wait=True, timeout=50)
train_dataset_proba_logistic_regression = scenario_1.train_dataset_proba_logistic_regression.read()
print(train_dataset_proba_logistic_regression.shape)
threshold_logistic_regression = scenario_1.threshold_logistic_regression.read()
print(threshold_logistic_regression)

In [ ]:
task = scenario_1.find_best_threshold_random_forest
task.submit(wait=True, timeout=50)
train_dataset_proba_random_forest = scenario_1.train_dataset_proba_random_forest.read()
print(train_dataset_proba_random_forest.shape)
threshold_random_forest = scenario_1.threshold_random_forest.read()
print(threshold_random_forest)

In [ ]:
task = scenario_1.find_best_threshold_xgboost
task.submit(wait=True, timeout=50)
train_dataset_proba_xgboost = scenario_1.train_dataset_proba_xgboost.read()
print(train_dataset_proba_xgboost.shape)
threshold_xgboost = scenario_1.threshold_xgboost.read()
print(threshold_xgboost)

### TASKS roc

In [ ]:
task = scenario_1.task_roc_baseline
task.submit(wait=True, timeout=50)
roc_data_baseline = scenario_1.roc_data_baseline.read()
print(roc_data_baseline.shape)
auc_score_baseline = scenario_1.auc_score_baseline.read()
print(auc_score_baseline)

In [ ]:
task = scenario_1.task_roc_logistic_regression
task.submit(wait=True, timeout=50)
roc_data_logistic_regression = scenario_1.roc_data_logistic_regression.read()
print(roc_data_logistic_regression.shape)
auc_score_logistic_regression = scenario_1.auc_score_logistic_regression.read()
print(auc_score_logistic_regression)

In [ ]:
task = scenario_1.task_roc_random_forest
task.submit(wait=True, timeout=50)
roc_data_random_forest = scenario_1.roc_data_random_forest.read()
print(roc_data_random_forest.shape)
auc_score_random_forest = scenario_1.auc_score_random_forest.read()
print(auc_score_random_forest)

In [ ]:
task = scenario_1.task_roc_xgboost
task.submit(wait=True, timeout=50)
roc_data_xgboost = scenario_1.roc_data_xgboost.read()
print(roc_data_xgboost.shape)
auc_score_xgboost = scenario_1.auc_score_xgboost.read()
print(auc_score_xgboost)

### TASKS create_metrics_on_train

In [ ]:
task = scenario_1.create_metrics_on_train_baseline
task.submit(wait=True, timeout=50)
train_dataset_metrics_baseline = scenario_1.train_dataset_metrics_baseline.read()
print(train_dataset_metrics_baseline.shape)

In [ ]:
task = scenario_1.create_metrics_on_train_logistic_regression
task.submit(wait=True, timeout=50)
train_dataset_metrics_logistic_regression = scenario_1.train_dataset_metrics_logistic_regression.read()
print(train_dataset_metrics_logistic_regression.shape)

In [ ]:
task = scenario_1.create_metrics_on_train_random_forest
task.submit(wait=True, timeout=50)
train_dataset_metrics_random_forest = scenario_1.train_dataset_metrics_random_forest.read()
print(train_dataset_metrics_random_forest.shape)


In [ ]:
task = scenario_1.create_metrics_on_train_xgboost
task.submit(wait=True, timeout=50)
train_dataset_metrics_xgboost = scenario_1.train_dataset_metrics_xgboost.read()
print(train_dataset_metrics_xgboost.shape)

### TASKS task_create_metrics

In [ ]:
task = scenario_1.task_create_metrics_baseline
task.submit(wait=True, timeout=1)
metrics_baseline = scenario_1.metrics_baseline.read()
for key, value in metrics_baseline.items(): print(f'{key}: {value}')

In [ ]:
task = scenario_1.task_create_metrics_logistic_regression
task.submit(wait=True, timeout=1)
metrics_logistic_regression = scenario_1.metrics_logistic_regression.read()
for key, value in metrics_logistic_regression.items(): print(f'{key}: {value}')

In [ ]:
task = scenario_1.task_create_metrics_random_forest
task.submit(wait=True, timeout=1)
metrics_random_forest = scenario_1.metrics_random_forest.read()
for key, value in metrics_random_forest.items(): print(f'{key}: {value}')

In [ ]:
task = scenario_1.task_create_metrics_xgboost
task.submit(wait=True, timeout=1)
metrics_xgboost = scenario_1.metrics_xgboost.read()
for key, value in metrics_xgboost.items(): print(f'{key}: {value}')

### TASKS task_create_results

In [ ]:
task = scenario_1.task_create_results_baseline
task.submit(wait=True, timeout=1)
results_baseline = scenario_1.results_baseline.read()
print(results_baseline.shape)

In [ ]:
task = scenario_1.task_create_results_logistic_regression
task.submit(wait=True, timeout=1)
results_logistic_regression = scenario_1.results_logistic_regression.read()
print(results_logistic_regression.shape)

In [ ]:
results_logistic_regression

In [ ]:
task = scenario_1.task_create_results_random_forest
task.submit(wait=True, timeout=1)
results_random_forest = scenario_1.results_random_forest.read()
print(results_random_forest.shape)

In [ ]:
task = scenario_1.task_create_results_xgboost
task.submit(wait=True, timeout=1)
results_xgboost = scenario_1.results_xgboost.read()
print(results_xgboost.shape)

In [ ]:
# scenarios = tp.get_entities_by_config_id(tp.get_scenarios()[0])
# print(scenarios)
# scenario = scenarios[0]

In [ ]:
# print_object_attributes(scenario)

In [ ]:
# print(scenario.get_inputs())
# print(scenario.get_outputs())

In [ ]:
# input_nodes = scenario.get_inputs()
# output_nodes = scenario.get_outputs()

In [ ]:
# input_node_1 = list(input_nodes)[0]
# output_node_1 = list(output_nodes)[0]
# # output_node_1 = list(output_node_1)[0].read()


In [ ]:
# list(output_nodes)[0]
# print_object_attributes(list(output_nodes)[0])

In [ ]:
# for o_n in list(output_nodes):
#     # print(o_n._config_id )
#     if o_n._config_id.startswith('results_'): 
#         print(o_n._config_id)

# # o_n_1.read()

In [ ]:
# print_object_attributes(input_node_1)

In [ ]:
# print_object_attributes(output_node_1)

In [ ]:
# for i in list(outputs):
#     a=i.read()
#     print(type(a))
    

In [ ]:
# a = tp.get_entities_by_config_id(list(scenario_1.data_nodes)[0])

In [ ]:
# a[0].read()

In [ ]:
# for i in scenario_1.get_outputs(): print(i)

In [ ]:
# list(scenario_1.data_nodes.keys())

In [ ]:
# print([(s.name, s.get_outputs().read()) for s in tp.get_scenarios()])